In [1]:
import tensorflow.compat.v1 as tf
import tf_slim as slim

In [5]:
def inspect_model(checkpoint_path):
    reader = tf.train.load_checkpoint('vggish_model.ckpt')
    shape_from_key = reader.get_variable_to_shape_map()
    dtype_from_key = reader.get_variable_to_dtype_map()
    return shape_from_key, dtype_from_key

In [2]:
reader = tf.train.load_checkpoint('vggish_model.ckpt')

In [9]:
reader.get_tensor("vggish/conv1/biases")

array([-0.04159481, -0.01429164, -0.38100675, -0.10166927, -0.03815449,
       -0.15701427, -0.07046445, -0.23474823, -0.0998854 , -0.09906989,
       -0.20672537, -0.3738388 , -0.01119703, -0.04835688, -0.2237279 ,
       -0.20798603, -0.00271479, -0.03359881, -0.01850887, -0.29751638,
       -0.14957175,  0.00272396, -0.06416758, -0.20704597, -0.02230643,
       -0.27596423, -0.07822472,  0.00103446, -0.08780743, -0.04968313,
       -0.16092052, -0.15638474, -0.0593946 , -0.21262135, -0.2360925 ,
       -0.24887156, -0.38017526, -0.12486155,  0.00057994,  0.00158324,
       -0.08591457, -0.22833213, -0.14211905, -0.14730002, -0.5686185 ,
       -0.1723926 , -0.01410491, -0.29759297, -0.26884976, -0.09076596,
       -0.17841725, -0.25356275, -0.06617288, -0.12007098, -0.02347378,
       -0.3551823 , -0.16344738, -0.2631998 , -0.43284968, -0.1120552 ,
       -0.12158959, -0.33472678,  0.00334119, -0.03985222], dtype=float32)

In [3]:
shape_from_key = reader.get_variable_to_shape_map()
dtype_from_key = reader.get_variable_to_dtype_map()

In [4]:
sorted(shape_from_key.keys())

['vggish/conv1/biases',
 'vggish/conv1/weights',
 'vggish/conv2/biases',
 'vggish/conv2/weights',
 'vggish/conv3/conv3_1/biases',
 'vggish/conv3/conv3_1/weights',
 'vggish/conv3/conv3_2/biases',
 'vggish/conv3/conv3_2/weights',
 'vggish/conv4/conv4_1/biases',
 'vggish/conv4/conv4_1/weights',
 'vggish/conv4/conv4_2/biases',
 'vggish/conv4/conv4_2/weights',
 'vggish/fc1/fc1_1/biases',
 'vggish/fc1/fc1_1/weights',
 'vggish/fc1/fc1_2/biases',
 'vggish/fc1/fc1_2/weights',
 'vggish/fc2/biases',
 'vggish/fc2/weights',
 'vggish/logits/biases',
 'vggish/logits/weights']

In [6]:
a, b = inspect_model('vggish_model.ckpt')

In [13]:
import torch

In [20]:
conv1 = torch.nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3,3), stride=1, padding='same')

In [26]:
conv1(torch.rand(30, 1, 96, 64))

tensor([[[[ 5.0285e-04,  9.9504e-02, -4.0951e-03,  ..., -6.4867e-02,
            1.1492e-02, -7.3017e-02],
          [-5.8164e-02, -3.7027e-02, -3.0985e-01,  ..., -3.8618e-01,
            5.2313e-02,  5.0173e-02],
          [-2.7349e-01, -1.8926e-02, -1.0884e-03,  ..., -2.9540e-01,
           -3.7926e-01, -2.3913e-01],
          ...,
          [-1.6717e-01, -3.0725e-02, -2.2245e-01,  ..., -2.2266e-01,
           -4.4004e-01, -1.6834e-01],
          [-1.4153e-01, -3.6451e-01, -1.4172e-01,  ..., -2.7254e-02,
           -2.0336e-01, -1.6563e-01],
          [-3.6479e-01, -2.7727e-01, -3.0480e-01,  ..., -2.6107e-01,
           -4.6556e-01, -4.9207e-01]],

         [[-6.1238e-02, -9.1456e-02, -3.3228e-01,  ..., -2.1719e-01,
            6.0241e-02,  1.1456e-02],
          [-2.9322e-01,  8.6207e-03,  9.2305e-02,  ..., -1.5170e-01,
           -1.2623e-01, -2.1448e-01],
          [-5.8887e-02, -6.8017e-02,  1.5635e-01,  ...,  4.8318e-02,
           -1.4664e-01, -6.5657e-02],
          ...,
     

In [40]:
conv1.weight = torch.nn.Parameter(torch.tensor(reader.get_tensor("vggish/conv1/weights").T))
conv1.bias = torch.nn.Parameter(torch.tensor(reader.get_tensor("vggish/conv1/biases")))

In [73]:
class VGGish(torch.nn.Module):
    def __init__(self):
        super(VGGish, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3,3), stride=1, padding='same')
        self.pool1 = torch.nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding='same')
        self.pool2 = torch.nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv3_1 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=1, padding='same')
        self.conv3_2 = torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=1, padding='same')
        self.pool3 = torch.nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv4_1 = torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride=1, padding='same')
        self.conv4_2 = torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding='same')
        self.pool4 = torch.nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.flatten = torch.nn.Flatten() 
        self.fc1 = torch.nn.Linear(12288, 4096)
        self.fc2 = torch.nn.Linear(4096, 4096)
        self.fc3 = torch.nn.Linear(4096, 128)
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.relu(self.conv3_1(x))
        x = self.relu(self.conv3_2(x))
        x = self.pool3(x)
        x = self.relu(self.conv4_1(x))
        x = self.relu(self.conv4_2(x))
        x = self.pool4(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [74]:
model = VGGish()

In [75]:
model.

torch.Size([30, 128])

In [48]:
encoder_output = torch.rand(128).numpy(force=True)

In [49]:
encoder_output.shape

(128,)

In [61]:
encoder_output.reshape(-1,1,1) # (infer, 1, 1)

array([[[0.17850667]],

       [[0.14358383]],

       [[0.23835939]],

       [[0.03121907]],

       [[0.51520836]],

       [[0.06090778]],

       [[0.03508103]],

       [[0.31607258]],

       [[0.41456878]],

       [[0.49675465]],

       [[0.03528702]],

       [[0.1830672 ]],

       [[0.59165424]],

       [[0.21781367]],

       [[0.8796475 ]],

       [[0.3546446 ]],

       [[0.270858  ]],

       [[0.21400458]],

       [[0.5919878 ]],

       [[0.98443305]],

       [[0.10747987]],

       [[0.54591763]],

       [[0.75769675]],

       [[0.4325629 ]],

       [[0.9683805 ]],

       [[0.82379675]],

       [[0.4239357 ]],

       [[0.07768202]],

       [[0.9482044 ]],

       [[0.28108084]],

       [[0.2376377 ]],

       [[0.9165034 ]],

       [[0.05837196]],

       [[0.39545172]],

       [[0.21329302]],

       [[0.4396729 ]],

       [[0.40198892]],

       [[0.6954931 ]],

       [[0.8155821 ]],

       [[0.2924174 ]],

       [[0.25189835]],

       [[0.29511